# Benchmarks and Tests of Maps for Single-Element Substrates

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Pt(111) Surface

Generate the ASE.Atoms instance using the fcc111 build function. 

In [ ]:
from ase.build import fcc111
Pt111 = fcc111("Pt",size=(4,4,3),a=3.94,orthogonal=True,periodic=True,vacuum = 10)

From the `ASE.Cell` and the generated `ASE.Atoms`, create a `MapSy.Grid` and a `MapSy.System`

In [ ]:
from mapsy.data import Grid
grid: Grid = Grid(cell=Pt111.cell)

In [ ]:
from mapsy.data import System
system: System = System(grid, Pt111, dimension=2, axis=2)

In [ ]:
from mapsy.io.parser import ContactSpaceGenerator, ContactSpaceModel
contactspacesettings = ContactSpaceModel.parse_obj({"mode": "system", "distance": 3.5, "spread": 1.0, "cutoff": 80, "threshold": 0.54})
contactspacesettings = ContactSpaceModel.parse_obj({"mode": "ionic", "cutoff": 20, "spread" : 1.0, "threshold": 0.4})
contactspace = ContactSpaceGenerator(contactspacesettings).generate(system)

In [ ]:
contactspace.data

In [ ]:
from mapsy.symfunc.input import SymmetryFunctionsModel, SymFuncModel
from mapsy.symfunc.parser import SymmetryFunctionsParser
symfuncsettings = SymmetryFunctionsModel.parse_obj({"functions": [SymFuncModel.parse_obj({"type":"ac","radius":4.5,"order":10,"compositional":False,"structural":True}),SymFuncModel.parse_obj({"type":"ac","radius":4.5,"order":10,"compositional":False,"structural":True,"radial":False})]})
symmetryfunctions = SymmetryFunctionsParser(symfuncsettings).parse()

In [ ]:
from mapsy.maps import Maps
maps = Maps(system,symmetryfunctions,contactspace)

In [ ]:
data = maps.atcontactspace()

We can visualize features to check how they look using `Maps.plot(feature: str)` or `Maps.plot(index: int)`. NOTE: to get the top face of the slab, we need to select `region=1`. 

In [ ]:
fig, axes = maps.scatter(index=4, cmap='Spectral', region=0, splitby='z', set_aspect='scaled')
plt.show()

## Dimensionality Reduction (PCA)

For visualization and post-processing purposes, perform dimensionality reduction on the generated features. Three components are useful for 2D and 3D plots.

In [ ]:
maps.reduce(4)

We can visually inspect how the PCAs correlate with the Cartesian coordinates of the points (e.g., PCA3 distinguishes between HCP and FCC hollow sites)

In [ ]:
maps.scatter(feature='pca1', axes=['x', 'y'],cmap='Spectral', splitby='z', region=1, set_aspect='scaled')
plt.show()

We can also verify how the contact space is transformed (folded) in the symmetry function space. 

In [ ]:
maps.scatter(feature='pca2', cmap='Spectral', axes=['pca0','pca1'], region=1,alpha=1., set_aspect='equal')
plt.show()

## Perform Clustering on Generated Features

Use SpectralClustering to find N clusters in the featured data. 

In [ ]:
maps.reduce(6)

In [ ]:
features = [f"pca{i}" for i in range(1,6)]
print(features)

In [ ]:
maps.cluster(nclusters=15, features=[f"pca{i}" for i in range(1,6)])

Given the clusters, plot the connectivity matrix

In [ ]:
plt.matshow(maps.cluster_edges)

Given the clusters and the connectivity, find the high-symmetry sites

In [ ]:
maps.sites(region=1)

Visualize the results

In [ ]:
fix, ax = maps.scatter(feature='Cluster', region=1, categorical=True, axes=['pca0','pca3'], alpha=0.1,  set_aspect='scaled')
plt.show()

In [ ]:
fig, ax = maps.scatter(feature='Cluster', region=1, categorical=True, alpha=0.5, s=20, splitby='z',set_aspect='scaled', centroids=True)
plt.show()

In [ ]:
axes = ['pca0','pca1']
fig, axs = maps.scatter(feature='Cluster', categorical=True, axes=axes, alpha=0.05, s=20, set_aspect='scaled')

import networkx as nx
G = nx.from_numpy_array(maps.cluster_edges,create_using=nx.DiGraph,parallel_edges=False)
pos = maps.data.loc[maps.centroids,axes].values
weights = [ d['weight']/500 for (u, v, d) in G.edges(data=True)]
axs = [axs]
for ax in axs:
    nx.draw(G, pos, width=weights, ax=ax, alpha=0.4)
    limits=ax.axis('on') # turns on axis
    ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
plt.show()

## Pt(100) Surface

In [ ]:
from ase.build import fcc100
Pt100 = fcc100("Pt",size=(4,4,3),a=3.94,orthogonal=True,periodic=True,vacuum = 10)

In [ ]:
from mapsy.data import Grid
grid100: Grid = Grid(cell=Pt100.cell)

In [ ]:
from mapsy.data import System
system100: System = System(grid100, Pt100, dimension=2, axis=2)

In [ ]:
from mapsy.io.parser import ContactSpaceGenerator, ContactSpaceModel
contactspacesettings = ContactSpaceModel.parse_obj({"mode": "system", "distance": 3.5, "spread": 1.0, "cutoff": 80, "threshold": -1})
contactspace100 = ContactSpaceGenerator(contactspacesettings).generate(system100)

In [ ]:
contactspace100.data

In [ ]:
from mapsy.maps import Maps
maps100 = Maps(system100,symmetryfunctions,contactspace100)

In [ ]:
data100 = maps100.atcontactspace()

In [ ]:
maps100.plot(feature='ACSF_RS_r4.5_003', cmap='Spectral', levels=24, region=1, set_aspect='scaled')
plt.show()

In [ ]:
maps100.reduce(4)

In [ ]:
maps100.cluster(nclusters=6)

In [ ]:
plt.matshow(maps100.cluster_graph)
plt.colorbar()
plt.matshow(maps100.cluster_edges)
plt.colorbar()

In [ ]:
maps100.sites(region=1)

In [ ]:
axes = ['x','y']#['pca0','pca2']
maps100.scatter(feature='Cluster', region=1, axes=axes, categorical=True, alpha=0.95, s=20, centroids=True, set_aspect='scaled')
plt.show()

In [ ]:
axes = ['pca0','pca1']
fig, ax = maps100.scatter(feature='Cluster', region=1, categorical=True, axes=axes, alpha=1, s=10, set_aspect='scaled')

import networkx as nx
G = nx.from_numpy_array(maps100.cluster_edges,create_using=nx.DiGraph,parallel_edges=False)
pos = maps100.data.loc[maps100.centroids,axes].values
weights = [ d['weight']/500 for (u, v, d) in G.edges(data=True)]
nx.draw(G, pos, width=weights, ax=ax, alpha=0.4)
limits=ax.axis('on') # turns on axis
ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
plt.show()

## Pt(110) Surface

In [ ]:
from ase.build import fcc110
Pt110 = fcc110("Pt",size=(4,4,3),a=3.94,orthogonal=True,periodic=True,vacuum = 10)

In [ ]:
from mapsy.data import Grid
grid110: Grid = Grid(cell=Pt110.cell)

In [ ]:
from mapsy.data import System
system110: System = System(grid110, Pt110, dimension=2, axis=2)

In [ ]:
from mapsy.io.parser import ContactSpaceGenerator, ContactSpaceModel
contactspacesettings = ContactSpaceModel.parse_obj({"mode": "system", "distance": 3.5, "spread": 1.0, "cutoff": 80, "threshold": -1})
contactspace110 = ContactSpaceGenerator(contactspacesettings).generate(system110)

In [ ]:
contactspace110.data

In [ ]:
from mapsy.maps import Maps
maps110 = Maps(system110,symmetryfunctions,contactspace110)

In [ ]:
data110 = maps110.atcontactspace()

In [ ]:
fig, ax = maps110.plot(index=7, cmap='Spectral', levels=24, region=1, set_aspect='scaled')
plt.show()

In [ ]:
maps110.reduce(4)

In [ ]:
maps110.cluster(nclusters=6)

In [ ]:
plt.matshow(maps110.cluster_edges)
plt.colorbar()

In [ ]:
maps110.sites(region=1)

In [ ]:
axes = ['x','y']#['pca0','pca2']
maps110.scatter(feature='Cluster', region=1, axes=axes, categorical=True, alpha=0.95, s=20, centroids=True, set_aspect='scaled')
plt.show()

In [ ]:
axes = ['pca0','pca2']
fig, ax = maps110.scatter(feature='Cluster', region=1, categorical=True, axes=axes, alpha=0.4, s=20)

ax.set_xlabel('Principal Component 0', fontsize=15)
ax.set_ylabel('Principal Component 1', fontsize=15)
import networkx as nx
G = nx.from_numpy_array(maps110.cluster_edges,create_using=nx.DiGraph,parallel_edges=False)
pos = maps110.data.loc[maps110.centroids,axes].values
weights = [ d['weight']/500 for (u, v, d) in G.edges(data=True)]
nx.draw(G, pos, width=weights, ax=ax, alpha=0.4)
limits=ax.axis('on') # turns on axis
ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
plt.show()